In [ ]:
!pip install transformers torch accelerate bitsandbytes accelerate bitsandbytes

In [1]:
!nvidia-smi

Mon Apr 17 03:10:37 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.46                 Driver Version: 531.61       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2070         On | 00000000:26:00.0  On |                  N/A |
|  0%   55C    P0               49W / 185W|    712MiB /  8192MiB |      1%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import torch

In [3]:
# 8bit mode.  Turns out my little old video card cannot handle 16bit mode of even the smallest version.

from transformers import AutoModelForCausalLM, pipeline, AutoTokenizer
import time

model = AutoModelForCausalLM.from_pretrained(
    "databricks/dolly-v2-3b",
    device_map="auto",
    torch_dtype=torch.float16,
    load_in_8bit=True)

tokenizer = AutoTokenizer.from_pretrained("databricks/dolly-v2-3b")

def prompt(s):
    input_ids = tokenizer(s, return_tensors="pt").input_ids.to('cuda')
    start = time.time()
    gen_tokens = model.generate(
      input_ids,
      do_sample=True,
      temperature=0.9,
      max_length=500
    )

    gen_text = tokenizer.batch_decode(gen_tokens)[0]
    end = time.time()
    print("time to generate: ", end - start)
    return gen_text


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda121.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 121
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda121.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib'), PosixPath('/usr/local/nvidia/lib64')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('["/tmp/vscode-ssh-auth-1a7ff5f6-4ff5-4fb3-ba84-38cffcc445aa.sock","/root/.gnupg/S.gpg-agent"]')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {

In [4]:
print(prompt("Who was the first President of the United States?"))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/models/gpt_neox/modeling_gpt_neox.py:219: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at ../aten/src/ATen/native/TensorCompare.cpp:493.)
  attn_scores = torch.where(causal_mask, attn_scores, mask_value)


time to generate:  128.60850191116333
Who was the first President of the United States?

Alexander Hamilton

Alexander Hamilton was a major figure in the political and intellectual life of the young Federal Republic of Germany in the 18th century. He was born in New York City, the oldest of the three sons of Jewish immigrants from the southern city of Stuttgart. His father, Isaac de Haan, had settled there in 1730 after emigrating from Prussia, where he had been a prosperous merchant and member of the Lutheran Church.

Alexander Hamilton grew up in poverty, but he showed exceptional ability. He read widely and showed early signs of great intellectual curiosity, a sharp mind, and a flair for languages. When he was fifteen years old, his father died, and, moved by his distress, the town elders decided to fund his university studies abroad.

Alexander Hamilton went to the University of Göttingen, where he enrolled as a law student in 1748. He distinguished himself as a member of the Unive

In [6]:
bigprompt = """
The president of the United States is the head of state and head of government of the United States, indirectly elected to a 
four-year term via the Electoral College.  The officeholder leads the executive branch of the federal government and is the 
commander-in-chief of the United States Armed Forces. Since the office was established in 1789, 45 men have served in 46 
presidencies. The first president, George Washington, won a unanimous vote of the Electoral College; one, Grover Cleveland, 
served two non-consecutive terms and is therefore counted as the 22nd and 24th president of the United States, giving rise to 
the discrepancy between the number of presidencies and the number of persons who have served as president. The incumbent 
president is Joe Biden.

Q: Who was the first President of the United States?

A: 
"""

print(prompt(bigprompt))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


time to generate:  85.29081439971924

The president of the United States is the head of state and head of government of the United States, indirectly elected to a 
four-year term via the Electoral College.  The officeholder leads the executive branch of the federal government and is the 
commander-in-chief of the United States Armed Forces. Since the office was established in 1789, 45 men have served in 46 
presidencies. The first president, George Washington, won a unanimous vote of the Electoral College; one, Grover Cleveland, 
served two non-consecutive terms and is therefore counted as the 22nd and 24th president of the United States, giving rise to 
the discrepancy between the number of presidencies and the number of persons who have served as president. The incumbent 
president is Joe Biden.

Q: Who was the first President of the United States?

A: 

George Washington

Q: George Washington won a unanimous vote of the Electoral College?

A: Yes, he did.


Q: One man served two non

In [ ]:
from langchain.llms import HuggingFacePipeline

local_llm = HuggingFacePipleine(pipeline=generate_text)